<a href="https://colab.research.google.com/github/shahmohamadi/IntermediateAdvancedPythonExamples/blob/master/RandomForestt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://github.com/shawn-terryah/Twitter_Geolocation#step-4-train-a-meta-level-random-forest-classifier
# https://stackabuse.com/seaborn-library-for-data-visualization-in-python-part-1/
# https://towardsdatascience.com/improving-random-forest-in-python-part-1-893916666cd

!unzip /content/no_meta_df.zip
!unzip /content/yes_meta_df.zip
!unzip /content/control_meta_df.zip

In [1]:
!pip install demoji

In [ ]:
# nan_value = float("NaN")
# yes_df.replace("", nan_value, inplace=True)
# yes_df.dropna(subset = ["full_text"], inplace=True)
# # yes_df = yes_df.sample(frac=0.1, replace=True, random_state=1)
# print('new yes df shape: ', yes_df.shape)

# no_df.replace("", nan_value, inplace=True)
# no_df.dropna(subset = ["full_text"], inplace=True)

# no_df = no_df.sample(n=77447)
# # no_df = no_df.sample(frac=0.1, replace=True, random_state=1)
# print('new no df shape: ', no_df.shape)

# control_df = control_df.sample(n=77447)
# control_df.replace("", nan_value, inplace=True)
# control_df.dropna(subset = ["full_text"], inplace=True)

# df = yes_df.append(control_df)
# df['clean_text'] = df['full_text'].apply(lambda x: preprocess_text(x))
# print('main df shape', df.shape)

In [ ]:
from numpy import mean
from numpy import std
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import ast 
import pandas as pd
import numpy as np
import string
import json
import re
import os
import csv
from sklearn.metrics import roc_auc_score
from nltk.tag.sequential import ClassifierBasedPOSTagger 
from nltk.corpus import treebank 
from sklearn.ensemble import RandomForestClassifier
import demoji
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
nltk.download()
stop_words = set(stopwords.words('english'))
demoji.download_codes()

In [3]:
control_df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/control_meta_df4.pkl')
no_df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/no_meta_df4.pkl')
yes_df = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/yes_meta_df4.pkl')

print('control df: ', control_df.shape, '\nyes df: ', yes_df.shape, '\nno df: ', no_df.shape)

control df:  (190158, 15) 
yes df:  (77446, 15) 
no df:  (95773, 15)


In [ ]:
def preprocess_text(tweet):
    tweet = str(tweet)
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'\@\w+|\#\w+', '', tweet)
    # tweet = re.sub(r'[@#]', '', tweet)
    # demoji.download_codes()
    tweet = demoji.replace(tweet, '')  # remove all emojis
    # tweet = emoji.demojize(tweet)
    # tweet = tweet.replace(":", " ")
    # tweet = tweet.replace("_", " ")
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
    # ps = PorterStemmer()
    # stemmed_words = [ps.stem(w) for w in filtered_words]
    # lemmatizer = WordNetLemmatizer()
    # lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]
    return " ".join(filtered_words)

In [26]:
control_df = control_df.sample(n=77446)
no_df = no_df.sample(n=77446)
df = yes_df.append(control_df)
# df['clean_text'] = df['full_text'].apply(lambda x: preprocess_text(x))
print('main df: ', df.shape)

main df:  (154892, 15)


In [27]:
df.label

0         1
1         1
2         1
3         1
4         1
         ..
11173     0
7637      0
66481     0
38589     0
159286    0
Name: label, Length: 154892, dtype: int64

In [29]:
# aggregation functions
agg_funcs = {'full_text' : lambda x: ' '.join(x),
             'statuses_count': np.max,
             'friends_count' :np.mean,
             'favourites_count' : np.mean,
             'retweet_count' : np.mean,
             'followers_count' : np.mean,
             'label' : np.sum}

# Groupby 'screen_name' and then apply the aggregation functions in agg_funcs
df = df.fillna('')

df = df.groupby(['screen_name']).agg(agg_funcs).reset_index()



In [31]:
df.label = df.label.apply(lambda x: 1 if x>0 else 0)

In [32]:
df.label

0      1
1      1
2      0
3      0
4      1
      ..
154    1
155    0
156    0
157    0
158    0
Name: label, Length: 159, dtype: int64

In [33]:
friends_count = df['friends_count'].values.reshape(df.shape[0], 1)
statuses_count = df['statuses_count'].values.reshape(df.shape[0], 1)
favourites_count = df['favourites_count'].values.reshape(df.shape[0], 1)
followers_count = df['followers_count'].values.reshape(df.shape[0], 1)

In [34]:
add_features = np.hstack((friends_count,statuses_count, favourites_count, followers_count))

In [ ]:
# add_features.shape
print(full_text_LR_decsfunc.shape, add_features.shape)
X_train.shape

# meta_X = np.hstack((full_text_LR_decsfunc, add_features))
# meta_X.shape

meta_X = np.hstack((full_text_LR_decsfunc, add_features))
این خط ارور دارد چون ابعاد متفاوت است.


print(full_text_LR_decsfunc.shape, add_features.shape)
X_train.shape

(116169,) (154892, 4)

(116169,)

به همین دلیل است که در کد نمونه هم اول دیتاست را جدا کرده بعد بقیه مراحل. چون نتایج مدل لاجستیک رگرششن مربوط به ۷۰ درصد داده ها است که تصادفی انتخاب شده و ما نمیدانیم کدام داده ها تا متا دیتای مربوطه را به آنها بچسبانیم 


In [ ]:
df['clean_text'] = df['full_text'].apply(lambda x: preprocess_text(x))

In [35]:
X_train, X_test, y_train, y_test = train_test_split(add_features, df['label'], random_state=0, shuffle=True)

In [37]:
# X_train = X_train.fillna(X_train.mean())

# vect = TfidfVectorizer(min_df=5).fit(X_train)  ## when using clean_text
# X_train_vectorized = vect.transform(X_train)

# vect = TfidfVectorizer(min_df=5).fit(X_train.values.astype('str'))  ## when using full_text
# X_train_vectorized = vect.transform(X_train.values.astype('str'))

model = LogisticRegression(max_iter=800)
# model.fit(X_train_vectorized, y_train)

# full_text_LR_decsfunc = model.decision_function(X_train_vectorized)  # for future use in RF

# predictions = model.predict(vect.transform(X_test))

# model = RandomForestClassifier(n_estimators=20, n_jobs=-1, random_state=0)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

y_pred = predictions
print('AUC of RF: ', roc_auc_score(y_test, predictions))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

AUC of RF:  0.7071611253196931
Mean Absolute Error: 0.275
Mean Squared Error: 0.275
Root Mean Squared Error: 0.5244044240850758


count vectorizer AUC of LR model:  0.8000839890550887 for yes/control on full_text


count vectorizer AUC of LR model:  0.7699083776501068 for yes/control on clean_text without emoji and on filtered words



count vectorizer AUC of LR model:  0.7088813242976063 for yes/no on full_text


count vectorizer AUC of LR model:  0.6822676596633498 for yes/no on clean_text without emoji and on filtered words

# random forest on 20 estimator
on yes/controm with unclean text:


Mean Absolute Error: 0.23063812204632905

Mean Squared Error: 0.23063812204632905

Root Mean Squared Error: 0.48024797974205896

AUC of RF:  0.7698903756887873

# on 50 estimator

AUC of RF:  0.7778957559166833

Mean Absolute Error: 0.22289078842031868

Mean Squared Error: 0.22289078842031868

Root Mean Squared Error: 0.47211310977383236


# on 100 estimator

AUC of RF:  0.7831501854664477

Mean Absolute Error: 0.21772589933631176

Mean Squared Error: 0.21772589933631176

Root Mean Squared Error: 0.46661107931157375

# on 500 esimator

AUC of RF:  0.785947674515556

Mean Absolute Error: 0.21501433256720812

Mean Squared Error: 0.21501433256720812

Root Mean Squared Error: 0.4636963797219126


# with adding meta features 

friends_count,statuses_count, favourites_count,followers_count

UC of RF:  1.0
Mean Absolute Error: 0.0
Mean Squared Error: 0.0
Root Mean Squared Error: 0.0


# with meta features and group bying df on screen_name:

UC of RF:  0.7071611253196931

Mean Absolute Error: 0.275

Mean Squared Error: 0.275

Root Mean Squared Error: 0.5244044240850758


# AUC of LR on yes/control full_text

UC of RF:  0.8018956491925989

Mean Absolute Error: 0.19879658084342638

Mean Squared Error: 0.19879658084342638

Root Mean Squared Error: 0.4458661019223444

# AUC of LR on no/control full_twxt. so what does it mean?

UC of RF:  0.7843411862248555
Mean Absolute Error: 0.21491103478552798
Mean Squared Error: 0.21491103478552798
Root Mean Squared Error: 0.46358498119064206


# AUC of LR on control/control half and half labeled to 0/1 randomly 
# yes/yes and no/no daase seperatly produced same AUC

UC of RF:  0.49830601467019386

Mean Absolute Error: 0.5016010742691871

Mean Squared Error: 0.5016010742691871

Root Mean Squared Error: 0.7082380067951642

